# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
*   모든 구현은 [2. 데이터 전처리] 및 [3.모델 생성]에서만 진행
*   [4. 모델 저장]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *    트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
 *  모델 발표 자료 
* 제출 기한: **오후 6시 (단, 발표자료는 12시)**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2021_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 두개의 테스트 데이터 셋 분류 정확도에 대한 weighted sum
 *  model.evaluate(x_test1, y_test1) + model.evaluate(x_test2, y_test2) * 2
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예: 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성
 *  단, tensorflow.data 및 dataset API를 사용하기 위해 변경하는 것은 허용. 이 경우, model evaluation 파트도 해당 API를 쓰도록 변경  
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [2]:
# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.utils import np_utils
import numpy as np

# 데이터셋 로드 (Training dataset: CIFAR10, test dataset: CIFAR10 & CIFAR10의 변형)
(x_train, y_train), (x_test1, y_test1) = keras.datasets.cifar10.load_data()

test_ds2 = tfds.load('cifar10_1/v6', split='test', shuffle_files=False, batch_size=-1)
test_ds2 = tfds.as_numpy(test_ds2)
x_test2, y_test2 = test_ds2['image'], test_ds2['label']

#분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test1 = np_utils.to_categorical(y_test1)
y_test2 = np_utils.to_categorical(y_test2)

# 총 클래스 개수
num_classes = y_train.shape[1]
input_shape = x_train.shape[1:]

170508288/170498071 [==============================] - 5s 0us/step


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cifar10_1/v6/1.1.0.incompleteC88Q4K/cifar10_1-test.tfrecord


  0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset cifar10_1 downloaded and prepared to /root/tensorflow_datasets/cifar10_1/v6/1.1.0. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


# **2. 데이터 전처리**



In [3]:
# 데이터 전처리 (예: normalization)
# 원본 데이터와 전처리 후 데이터를 구분하기 위해, 변수명 x_train_after, x_test1_after, x_test2_after를 변경하지 말 것
x_train_after = x_train / 255.0
x_test1_after = x_test1 / 255.0
x_test2_after = x_test2 / 255.0

x_train_after= x_train_after.astype('float32')
x_test1_after= x_test1_after.astype('float32')
x_test2_after= x_test2_after.astype('float32')

In [4]:
batch_size =64
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True )
# color_mode : rgb / class_mode = 'categorical'
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
train_dataset = datagen.flow(x_train_after, y_train, batch_size=batch_size)

In [5]:
BATCH_SIZE =64

test1_dataset = tf.data.Dataset.from_tensor_slices((x_test1_after, y_test1))
test2_dataset = tf.data.Dataset.from_tensor_slices((x_test2_after, y_test2))

test1_dataset = test1_dataset.batch(BATCH_SIZE)
test2_dataset = test2_dataset.batch(BATCH_SIZE)


# **3. 모델 생성**

In [7]:
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D,Activation, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.callbacks import ModelCheckpoint

In [9]:
     #vgg16 내부구조 참고
     #vgg16에 없는 배치정규화를 추가하여 층을 구성하였고 vgg16의 conv층의 마지막 3-layers인 conv3-512를 제외
     #fine tuning의 경우  GlobalAveragePooling2D(), Dropout(), Dense(512), Dense(256), BatchNormalization를 섞어서 구성

     model = Sequential([ 
        Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),                 
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(128, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),

        Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),

        Conv2D(512, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(512, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(512, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),

        Conv2D(512, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(512, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(512, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),

        GlobalAveragePooling2D(),
        Dropout(0.5),
        #Flatten(), 

        Dense(512),
        BatchNormalization(),
        Activation('relu'),
      
        Dense(512),
        BatchNormalization(),
        Activation('relu'),
        
        Dense(256),
        BatchNormalization(),
        Activation('relu'),

        Dense(10, activation='softmax'),

        
     ])
 

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0

In [12]:
# model compile
from keras import optimizers
#'adam'
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
# model 체크포인트 생성
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/checkpoint_entire_best.h5', monitor='val_accuracy', verbose=1, save_weight_only=False, save_best_only=True, mode='auto')

In [14]:
# model early stpoing 설정
from keras.callbacks import EarlyStopping
earlystopping = EarlyStopping
earlystopping = EarlyStopping(monitor='val_loss',  
                              patience=10,         
                             )
# 모니터 기준 설정 (val loss) 
# 5회 Epoch동안 개선되지 않는다면 종료 (patience)

In [17]:
#model fit
model.fit(train_dataset, epochs = 60,  callbacks=[cp_callback, earlystopping], validation_data=(test2_dataset))

#x_train_after, y_train, (x_test2_after, y_test2)
#train_dataset / (test2_dataset)



Epoch 1/60
782/782 [==============================] - 39s 50ms/step - loss: 1.5364 - accuracy: 0.4240 - val_loss: 2.1023 - val_accuracy: 0.3780

Epoch 00001: val_accuracy improved from -inf to 0.37800, saving model to /content/checkpoint_entire_best.h5
Epoch 2/60
782/782 [==============================] - 38s 49ms/step - loss: 1.1541 - accuracy: 0.5917 - val_loss: 1.6087 - val_accuracy: 0.4825

Epoch 00002: val_accuracy improved from 0.37800 to 0.48250, saving model to /content/checkpoint_entire_best.h5
Epoch 3/60
782/782 [==============================] - 39s 50ms/step - loss: 0.9352 - accuracy: 0.6824 - val_loss: 1.5733 - val_accuracy: 0.5430

Epoch 00003: val_accuracy improved from 0.48250 to 0.54300, saving model to /content/checkpoint_entire_best.h5
Epoch 4/60
782/782 [==============================] - 38s 49ms/step - loss: 0.8137 - accuracy: 0.7289 - val_loss: 1.3081 - val_accuracy: 0.5595

Epoch 00004: val_accuracy improved from 0.54300 to 0.55950, saving model to /content/check

# **4. 모델 저장**

In [23]:
save_path = '/content/'
team_name = 'team20'

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_'+ team_name + '.h5')

# **5. 모델 로드 및 평가**

In [25]:
save_path = '/content/'
team_name = 'team20'

model = keras.models.load_model(save_path + 'model_entire_' + team_name + '.h5')

model.evaluate(x_test1_after, y_test1)
model.evaluate(x_test2_after, y_test2)

63/63 [==============================] - 1s 10ms/step - loss: 0.7779 - accuracy: 0.7810


[0.7778984904289246, 0.781000018119812]